In [1]:
#Init enviroment
!python3 -m venv OXTC
!source OXTC/bin/activate
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Generate the onnx model
%run generate_onnx_model.py

Creating model with in_size=5 and out_size=3


ValidationError: Nodes in a graph must be topologically sorted, however input 'x' of node: 
name:  OpType: MatMul
 is not output of any previous nodes.

In [3]:
#Import depenecys
import json
import tarfile
import pathlib
import tempfile
import numpy as np

import onnx
import tvm
from tvm import te
import tvm.relay as relay
from tvm.relay.backend import Executor, Runtime
import tvm.micro
import tvm.micro.testing
import tvm.contrib.utils
from tvm.micro import export_model_library_format

#Import model
onnx_model = onnx.load("simple_model_5_3.onnx")

In [4]:
RUNTIME = Runtime("crt", {"system-lib": True})
TARGET = tvm.micro.testing.get_target("crt")
EXECUTOR = tvm.relay.backend.Executor(
    "aot", {"unpacked-api": True, "interface-api": "c", "workspace-byte-alignment": 8}
)

input_name = "x"
shape_dict = {"x": (5,)}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

with tvm.transform.PassContext(opt_level=3, config={"tir.disable_vectorize": True}):
    module = tvm.relay.build(
        mod, target=TARGET, params=params, runtime=RUNTIME, executor=EXECUTOR
    )

Matmul is not optimized for x86. Recommend to use cblas/mkl/dnnl for better performance.


In [5]:
BUILD_DIR = pathlib.Path("./c_model")
BUILD_DIR.mkdir(exist_ok=True)
TAR_PATH = pathlib.Path(BUILD_DIR) / "model.tar"
export_model_library_format(module, TAR_PATH)

PosixPath('c_model/model.tar')

In [6]:
!tar -C ./c_model -xvf ./c_model/model.tar
!rm -r ./c_model/model.tar

./
./codegen/
./codegen/host/
./codegen/host/include/
./codegen/host/include/tvmgen_default.h
./codegen/host/src/
./codegen/host/src/default_lib0.c
./codegen/host/src/default_lib1.c
./metadata.json
./parameters/
./parameters/default.params
./src/
./src/default.relay
./runtime/
./runtime/CMakeLists.txt
./runtime/include/
./runtime/include/checksum.h
./runtime/include/dlpack/
./runtime/include/dlpack/dlpack.h
./runtime/include/dmlc/
./runtime/include/dmlc/any.h
./runtime/include/dmlc/array_view.h
./runtime/include/dmlc/base.h
./runtime/include/dmlc/blockingconcurrentqueue.h
./runtime/include/dmlc/build_config_default.h
./runtime/include/dmlc/common.h
./runtime/include/dmlc/concurrency.h
./runtime/include/dmlc/concurrentqueue.h
./runtime/include/dmlc/config.h
./runtime/include/dmlc/data.h
./runtime/include/dmlc/endian.h
./runtime/include/dmlc/filesystem.h
./runtime/include/dmlc/input_split_shuffle.h
./runtime/include/dmlc/io.h
./runtime/include/dmlc/json.h
./runtime/include/dmlc/logging.h